In [2]:
import os,sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
gparentdir = os.path.dirname(parentdir)

prototypedir = '/'.join([gparentdir,'prototyping']) 
minionsdir = '/'.join([gparentdir,'minions']) 

sys.path.insert(0, gparentdir) 
sys.path.insert(0, prototypedir) 
sys.path.insert(0, minionsdir) 

from prototyping.util import *

from DetermineRidges.RidgeAnalysis import PolylineGraphs,LabelledMesh

In [3]:
from scipy.spatial import distance

class Paths:
    def __init__( self,
                path: str = None,
                id: str = None,
                preprocessed: str = None,
                exp_path: str = None,
                tri_mesh: trimesh.Trimesh = None,
                selected_edges: dict = None) -> object:

        """
        A Mesh object contains a triangular 3D mesh.

        Args:
            path (str): String representing the path to the file.
            id (str): String representing the file id of the ply file.
            preprocessed (str): String representing the preprocessing stage of the ply file.
            exp_path (str): String representing the export folder where to save all derived data.
            mesh (trimesh.Trimesh): Trimesh.Trimesh object. 
            selected_vertices (dict): dictionary containing edge label (key) e.g. node ids in graph, and selection of vertex ids (value).
        """

        # check for None only to avoid warning messages in subclasses
        if path is not None:
            self.path = path
        if id is not None:
            self.id = id
        if preprocessed is not None:
            self.preprocessed = preprocessed
        if exp_path is not None:
            self.exp_path = exp_path
        if tri_mesh is not None:
            self.tri_mesh = tri_mesh
        if selected_edges is not None:
            self.selected_edges = selected_edges
          

    def shortest_paths(self):

        """
        Finds the shortest path between the preselected edges containing vertices on a triangular 3D mesh.

        Args:
            path (str): String representing the path to the file.
            id (str): String representing the file id of the ply file.
            preprocessed (str): String representing the preprocessing stage of the ply file.
            exp_path (str): String representing the export folder where to save all derived data.
            mesh (trimesh.Trimesh): Trimesh.Trimesh object. 
            selected_vertices (dict): dictionary containing edge label (key) e.g. node ids in graph, and selection of vertex ids (value).


        """        

        # edges without duplication
        edges = self.tri_mesh.edges_unique

        # the actual length of each unique edge
        length = self.tri_mesh.edges_unique_length

        # create the graph with edge attributes for length
        g = nx.Graph()
        for edge, L in zip(edges, length):
            g.add_edge(*edge, length=L)

        shortest_paths = {}
        # run the shortest path query using length for edge weight

        shortest_paths = {label:nx.shortest_path( g,
                                                            source=vertices[0],
                                                            target=vertices[1],
                                                            weight='length') 
                                for label,vertices in self.selected_edges.items()}
        
        shortest_paths_dist = {}
        
        # run the shortest path query using length for edge weight
        for key,ids in shortest_paths.items():
    
            shortest_paths_dist [key] = np.sum([distance.euclidean(self.tri_mesh.vertices[ids[n]],self.tri_mesh.vertices[ids[n+1]]) for n in range(len(ids)-1)])


        # for source,target in self.selected_edges:

        #     shortest_paths[(source,target)] = nx.shortest_path(g,
        #                         source=source,
        #                         target=target,
        #                         weight='length')
            
        self.shortest_paths = shortest_paths

        self.shortest_paths_dist = shortest_paths_dist


def geodesic_cut (mesh):

    mesh.tri_mesh.submesh()



def create_adjacent_label_submeshes (mesh,edges):

    """
    Creates submeshes of mesh according to passed labels. 
    
    Args:
        mesh (trimesh object): trimesh mesh object
        edges (dict): A dictionary containing edges (keys) and vertex ID's (values).             

    Returns: 
        submeshes (dict): A dictionary of submeshes of the labelled adjacency meshes.

    """ 
    
    selected_faces =   {   label:
                                    np.all(np.isin(mesh.faces,values), axis=1)
                                for label,values in edges.items()
                            }

    # print(selected_faces)
    # print(edges)

    submeshes = {   label:
                            mesh.submesh([selected_faces[label]])
                        for label in edges.keys()
                    }

    return submeshes

from prototyping.Functions.EssentialMeshAlteration import create_label_submeshes

from prototyping.Functions.EssentialLabelAlteration import  get_uniquelabel_vertlist

def find_edges_vertices(edges,labels):

    """
    Creates submeshes of mesh according to passed labels. 
    
    Args:
        mesh (trimesh object): trimesh mesh object
        edges (dict): A dictionary containing edges (keys) and vertex ID's (values).             

    Returns: 
        edge_vertices (dict): A dictionary of submeshes of the labelled adjacency meshes.

    """ 

    labels_vertices = get_uniquelabel_vertlist(labels)

    edges_vertices = {edge:[vert for label in edge for vert in labels_vertices[label]] for edge in edges}

    return edges_vertices



In [4]:
from prototyping.Functions.main import labelledmesh_procedures

preprocessed = '_GMOCF_r1.00_n4_v256.volume'
exp_path = 'exp_path'
id = str(207)
labelname = '_gt_labels'#'_Kmeans_labels'#
path = "/home/linsel/Documents/PhD/Data/Graphs/Fumane/CO/{}/".format(id)
ending = '.txt'

kwargs = {  'path': path,
            'id':id,
            'preprocessed':preprocessed,
            'labelname':labelname,
            'exp_path':exp_path
            }

method = 'label_slice'

LM = labelledmesh_procedures(method,**kwargs)

LM.get_NNs_ids()

In [5]:
from Classes.BasicClasses import manualEdges

ME = manualEdges()

ME.import_edges(path,id)

selected_edges = ME.manual_edges

In [6]:
edges_vertices = find_edges_vertices (selected_edges,LM.dict_label)
edges_vertices

# create_adjacent_label_submeshes (LM.tri_mesh,edges_vertices)
edges_submeshes = create_label_submeshes (LM.tri_mesh,edges_vertices)


In [7]:
edges_NNs = {(s,t):(LM.NNs_ids [s],LM.NNs_ids [t]) for s,t in selected_edges}

P = Paths(path,
      id,
      preprocessed,
      exp_path,
      LM.tri_mesh,
      edges_NNs)

# P.shortest_paths()
edges_NNs
# P.shortest_paths_dist

{(3, 4): (131587, 159158),
 (3, 1): (131587, 94378),
 (4, 6): (159158, 189791),
 (3, 10): (131587, 24901),
 (3, 16): (131587, 169373),
 (3, 13): (131587, 168955),
 (3, 19): (131587, 188993),
 (3, 25): (131587, 37020),
 (5, 10): (81449, 24901),
 (8, 9): (78357, 1945),
 (3, 22): (131587, 92808),
 (14, 13): (171828, 168955),
 (1, 9): (94378, 1945),
 (1, 12): (94378, 148419),
 (1, 18): (94378, 13338),
 (25, 19): (37020, 188993),
 (1, 15): (94378, 145631),
 (1, 21): (94378, 164046),
 (6, 5): (189791, 81449),
 (1, 24): (94378, 159354),
 (18, 19): (13338, 188993),
 (6, 23): (189791, 60977),
 (4, 5): (159158, 81449),
 (3, 12): (131587, 148419),
 (3, 18): (131587, 13338),
 (13, 21): (168955, 164046),
 (3, 15): (131587, 145631),
 (3, 21): (131587, 164046),
 (14, 15): (171828, 145631),
 (3, 24): (131587, 159354),
 (2, 1): (125061, 94378),
 (2, 7): (125061, 63713),
 (8, 17): (78357, 11751),
 (1, 11): (94378, 160780),
 (6, 1): (189791, 94378),
 (1, 8): (94378, 78357),
 (1, 14): (94378, 171828),
 (1

In [44]:
from Functions.BasicMSII1D import angle_between_vectors

def get_NNs_ids (tri_mesh,points):

    """
    Creates a dictionary containing labels (key) and nearest neighbor to label centroid as vertex id (values). 
    """

    # Nearest neighbour (NN) from label centroid to submesh        
    NNs_ids = {} 
                            
    for label, p in points.items():

        _,index = tri_mesh.kdtree.query(p)

        NNs_ids [label] = index

    return NNs_ids

def eigenvector (X):
    C = np.dot(X.T,X)
    eigval, eigvec = np.linalg.eigh(C)
    sortid = eigval.argsort()[::-1]

    eigvec = eigvec[:,sortid]

    return eigvec

def qsort(inlist):
    if inlist == []: 
        return []
    else:
        pivot = inlist[0]
        lesser = qsort([x for x in inlist[1:] if x < pivot])
        greater = qsort([x for x in inlist[1:] if x >= pivot])
        return lesser + [pivot] + greater

def mesh_transform_two_vertices(tri, verts):

    tri.vertices = tri.vertices - verts[0]

    verts[1] = verts[1] - verts[0]

    verts[0] = verts[0] - verts[0]

    # triverts = tri.vertices - tri.vertices[131587]
    dist = float(distance.euclidean(verts[0],verts[1]))

    _,trafo_mat = angle_between_vectors(np.array(verts[1]), 
                                        np.array([0,0,0]), 
                                        np.array([dist, 0,0])
                                        )

    trafo_mat = np.c_[np.c_[trafo_mat.T, np.array([0,0,0])].T,np.array([0,0,0,0])]

    tri.apply_transform(trafo_mat)    

def create_lines_graph(tri,vert_ids,lines):

    lines_set = {tuple(line[0]) for line in lines}

    [lines_set.add(tuple(line[1])) for line in lines]

    lines_list = []

    [lines_list.append(tuple(lin)) for line in lines for lin in line] 

    count = {lin:lines_list.count(lin) for _,lin in enumerate(lines_set)}

    print([lin for lin,c in count.items() if c == 1])

    lines_dict = {v:n for n, v in enumerate(lines_set)} 

    lines_dict_inv = {v: np.array(k) for k, v in lines_dict.items()}

    length = [distance.euclidean(line[0],line[1]) for line in lines]

    source = lines_dict[tuple(tri.vertices[vert_ids[0]])] #([n for n,line in enumerate(lines) if list(line[0]) == list(verts[0]))

    # lines_dict tri.vertices[vert_ids[1]]

    print(min([distance.euclidean(tri.vertices[vert_ids[1]],np.array(v)) for v in lines_dict.keys()]))

    target = lines_dict[tuple(tri.vertices[vert_ids[1]])] #= max([n for n,line in enumerate(lines) if list(line[0]) == list(tri.vertices[vert_ids[1]])])

    print(target)

    G = nx.Graph()
    for edge, L in zip(lines, length):

        edge_id = (lines_dict [tuple(edge[0])],lines_dict [tuple(edge[1])]) #(lines_dict [tuple(edge[0])],lines_dict [tuple(edge[1])])
        G.add_edge(*edge_id, length=L)    

    return G
    
def add_edges_from_faces (tri,G,face):

    for c in [(0,1),(1,2),(2,0)]:

        edge = tuple((face[c[0]],face[c[1]]))
        length = distance.euclidean (tri.vertices [edge[0]], tri.vertices [edge[1]])
        G.add_edge(*edge,length = length)    
    
    return G

def increase_path_size (tri,face_verts,selected_face_verts):

    for n,values in enumerate(face_verts):

        faces_list = list(tri.faces[np.any(np.isin(tri.faces,values), axis=1)])
        faces_list = [list(face) for face in faces_list]
        faces_list = [vert for vert in faces_list]

        [selected_face_verts.add (tuple(vert)) for vert in faces_list ]

        # [selected_face_verts.add (tuple(face)) for faces in faces_list for face in list(faces)]

        # for faces in faces_list:
        #     for face in faces:
        #         selected_face_verts.add (tuple(face))
            
    faces_dict = {tuple(face):n for n,face in enumerate(tri.faces)}

    selected_faces = {faces_dict[face] for face in selected_face_verts}

    return selected_face_verts,selected_faces

def shortest_distance(tri,vert_ids,selected_faces):

    G = nx.Graph()
    for face_id in selected_faces:

        face = tri.faces[face_id]

        G = add_edges_from_faces(tri,G,face)                                               


    shortest_paths = nx.shortest_path(G,
                                    source=vert_ids[0],                                        
                                    target=vert_ids[1],                                        
                                    weight='length')     


    shortest_paths_dist = np.sum([G.edges[(v,int(shortest_paths[n+1]))]['length'] for n,v in enumerate(shortest_paths[:-1])])

    return shortest_paths, shortest_paths_dist

def path_profile (self):
    
    tri = self.tri_mesh.copy()

    path_profiles = {}

    for edge_ids, vert_ids in self.selected_edges.items():

        verts = {0:tri.vertices[vert_ids[0]], 
                 1:tri.vertices[vert_ids[1]]}
        
        mesh_transform_two_vertices(tri, verts)

        lines,faces = trimesh.intersections.mesh_plane(tri,[0,1,0],[0,0,0],return_faces=True)

        # creates a line graph, which is not sufficient to build a complete cycle graph
        # create_lines_graph ()
        
        face_verts = set()
        G = nx.Graph()
        for face_id in faces:

            face = tri.faces[face_id]

            G = add_edges_from_faces(tri,G,face)

            [face_verts.add (vert) for vert in face]
 

        try:
            shortest_paths = nx.shortest_path(G,
                                            source=vert_ids[0],                                        
                                            target=vert_ids[1],                                        
                                            weight='length')     


            shortest_paths_dist = np.sum([G.edges[(v,int(shortest_paths[n+1]))]['length'] for n,v in enumerate(shortest_paths[:-1])])

        except:
            selected_face_verts = set()
            try:
                selected_face_verts,selected_faces = increase_path_size (tri,face_verts,selected_face_verts)
                shortest_distance(tri,vert_ids,selected_faces)
            except:
                try:
                    selected_face_verts,selected_faces = increase_path_size (tri,face_verts,selected_face_verts)
                    shortest_distance(tri,vert_ids,selected_faces)
                except:
                    print(edge_ids)


            # selected_face_verts = set()
            # for n,values in enumerate(face_verts):

            #     faces_list = list(tri.faces[np.any(np.isin(tri.faces,values), axis=1)])
            #     faces_list = [list(face) for face in faces_list]
            #     faces_list = [vert for vert in faces_list]

            #     [selected_face_verts.add (tuple(vert)) for vert in faces_list ]

            #     if n % 10000 == 0:
            #         print(n)

            #     # [selected_face_verts.add (tuple(face)) for faces in faces_list for face in list(faces)]

            #     # for faces in faces_list:
            #     #     for face in faces:
            #     #         selected_face_verts.add (tuple(face))
                    
            # faces_dict = {tuple(face):n for n,face in enumerate(tri.faces)}

            # selected_faces = {faces_dict[face] for face in selected_face_verts}
                                                        
            # G = nx.Graph()
            # for face_id in selected_faces:

            #     face = tri.faces[face_id]

            #     G = add_edges_from_faces(tri,G,face)

            #     {vert for vert in face} 

            # shortest_paths = nx.shortest_path(G,
            #                                 source=vert_ids[0],                                        
            #                                 target=vert_ids[1],                                        
            #                                 weight='length')     


            # shortest_paths_dist = np.sum([G.edges[(v,int(shortest_paths[n+1]))]['length'] for n,v in enumerate(shortest_paths[:-1])])


        path_profiles [edge_ids] =  {'path':shortest_paths,'dist':shortest_paths_dist}
        
        
        # cycle_length = [distance.euclidean(lines_dict_inv[line[0]],lines_dict_inv[line[1]]) for n,line in enumerate(g_cycle_edges)]

        # g_cycle = nx.Graph()
        # for edge, L in zip(g_cycle_edges, cycle_length):
        #     edge_id = (lines_dict_inv [edge[0]],lines_dict_inv [edge[1]]) #(lines_dict [tuple(edge[0])],lines_dict [tuple(edge[1])])
        #     g.add_edge(*edge_id, length=L)        

        # shortest_paths = {}
        # # run the shortest path query using length for edge weight

        # shortest_paths = nx.shortest_path(g,
        #                                   source=source,                                        
        #                                   target=target,                                        
        #                                   weight='length') 
        

        # simple_paths = nx.all_simple_paths (g, source, target, cutoff=20)

        # print()
        
        # # print([len(path) for path in nx.all_shortest_paths (g, source, target)])
        # for path in simple_paths:
        #     print(len(path)) 
        
        
        # run the shortest path query using length for edge weight
    
        # shortest_paths_dist = np.sum([g.edges[(v,int(shortest_paths[n+1]))]['length'] for n,v in enumerate(shortest_paths[:-1])])

        # 
        # print(shortest_paths_dist)

    return path_profiles

# P.selected_edges = {(3,4): tuple(NNs_ids.keys())}
# print(P.selected_edges)
path_profiles = path_profile(P)


(3, 4)
(3, 1)
(4, 6)
(3, 10)
(3, 16)
(3, 13)
(3, 19)
(3, 25)
(5, 10)
(8, 9)
(3, 22)
(14, 13)
(1, 9)
(1, 12)
(1, 18)
(25, 19)
(1, 15)
(1, 21)
(6, 5)
(1, 24)
(18, 19)
(6, 23)
(4, 5)
(3, 12)
(3, 18)
(13, 21)
(3, 15)
(3, 21)
(14, 15)
(3, 24)
(2, 1)
(2, 7)
(8, 17)
(1, 11)
(6, 1)
(1, 8)
(1, 14)
(1, 20)
(1, 17)
(1, 23)
(2, 22)
(6, 10)
(4, 1)
(3, 5)
(3, 11)
(6, 22)
(3, 8)
0
0
(3, 8)
(3, 14)
(4, 22)
(3, 20)
(10, 1)
(1, 7)
(1, 13)
(2, 6)
(1, 16)
(1, 19)
(1, 25)
(1, 22)


In [46]:
path_profiles[(3, 8)]['dist']

28.073741239021274

In [324]:
NNs_ids = LM.NNs_ids
edges_NNs = {(s,t):(NNs_ids [s],NNs_ids [t]) for s,t in selected_edges}

P = Paths(path,
      id,
      preprocessed,
      exp_path,
      LM.tri_mesh,
      edges_NNs)

P.shortest_paths()

P.shortest_paths_dist

{(3, 4): 8.036998161836785,
 (3, 1): 14.913753445191388,
 (4, 6): 6.38311097150588,
 (3, 10): 36.38599996566131,
 (3, 16): 14.331221135171997,
 (3, 13): 14.491625497918928,
 (3, 19): 11.204221634463613,
 (3, 25): 9.470511971063479,
 (5, 10): 3.460787081741411,
 (8, 9): 1.7450994468220045,
 (3, 22): 22.05240654417109,
 (14, 13): 2.4351315899152457,
 (1, 9): 20.174425269989293,
 (1, 12): 23.769783569659758,
 (1, 18): 19.353573420665185,
 (25, 19): 1.9836007559016606,
 (1, 15): 10.066607305180693,
 (1, 21): 15.362419119003563,
 (6, 5): 25.313411326476405,
 (1, 24): 9.65151576623194,
 (18, 19): 1.686758135623203,
 (6, 23): 22.665795148228284,
 (4, 5): 27.109611804747953,
 (3, 12): 17.312716053864275,
 (3, 18): 12.216052518442638,
 (13, 21): 2.563687053125109,
 (3, 15): 9.633792171758278,
 (3, 21): 16.898493153694993,
 (14, 15): 3.0230661775569128,
 (3, 24): 5.287609239939428,
 (2, 1): 13.032356257213484,
 (2, 7): 13.298887050095223,
 (8, 17): 1.7864775762521448,
 (1, 11): 25.35476676115747

In [321]:
NNs_ids
edges_NNs = {(3,4):(133298,104401)}

# 

P2 = Paths(path,
      id,
      preprocessed,
      exp_path,
      tri_line,
      edges_NNs)

P2.shortest_paths()

P2.shortest_paths_dist

{(3, 4): 8.178333014172308}

In [5]:
def shortest_path_procedure(obj,**kwargs):
    path = kwargs ['path'] 
    id = kwargs ['id']
    preprocessed = kwargs ['preprocessed']
    labelname = kwargs ['labelname']
    exp_path = kwargs ['exp_path'] 

    obj.load_labelled_mesh(path,id,preprocessed,labelname,exp_path) 

    obj.get_label_submeshes()

    obj.extract_ridges()

    # create node coordinates
    obj.get_centroids()


def shortest_path (obj,**kwargs):
    pass


def paths_procedures (method,**kwargs):
   
    LM = LabelledMesh()

    procedures = {'shortest_path':shortest_path_procedure}

    func = procedures.get(method)

    func(LM,**kwargs)

    return LM    

    # P = Paths()

    # return P




def Path_Procedures ():
LM = LabelledMesh()

LM.load_ply()

IndentationError: expected an indented block (3228525258.py, line 42)